In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
url = 'https://patient.info/forums/discuss/browse/genital-herpes-simplex-979'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}

In [ ]:
r = requests.get(url, headers = headers)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
threads = soup.find_all('div',{'class':'avt avt-sm'})

In [ ]:
full_list = []
next_page = soup.find_all('div',{'class':'group-discussions__paginate_bottom'})[0].find('option').text
page_count = int(next_page.split('/')[1])
print(page_count)
# groups_text_list


for page_cnt in range(page_count):
  print(page_cnt)
  main_url= 'https://patient.info/forums/discuss/browse/genital-herpes-simplex-979'
  current_url = main_url+"?page=%d" %(page_cnt)+'#group-discussions'
  print(current_url)
  r_current_url = requests.get(current_url, headers= headers)
  current_url_soup = BeautifulSoup(r_current_url.text, 'html.parser')
  current_url_threads = current_url_soup.find_all('div',{'class':'avt avt-sm'})
    
  for item in current_url_threads:
    temp_list=[]
    ## Extracting profile names
    profile_name=item.find('a',{'aria-label':'profile'}).text
    profile_name_cleaned = profile_name.replace('\r\n','').replace(" ","")
        #     thread_starter_name_list.append(profile_name_cleaned)
    print(profile_name_cleaned)
        ## Extracting urls of user_names
    profile_url = item.find('a',{'aria-label':'profile'})['href']
    profile_url_complete = "https://patient.info" + profile_url
        #     profile_url_list.append(profile_url_complete)
        
        #identifying groups in which a member is enrolled

    ###date the user joined:
    r0 = requests.get(profile_url_complete, headers= headers)
    soup0 = BeautifulSoup(r0.text, 'html.parser')
    
    join_date = soup0.find_all('span',{'class':'label masthead__actions__link'})[0].text.replace\
    ('\n\n\n\n\n\r\n                ', '').replace('\r\n            ','')
    cleaned_join_date= join_date.split(' ')[1]
    print('cleaned_join_date',cleaned_join_date)

    ###total_posts by the user till date
    total_posts = soup0.find_all('span',{'class':'label masthead__actions__link'})[1].text.replace('\n\n\n\n\n\r\n                ', '').replace('\r\n            ','')
    print('total_posts ', total_posts)    

    ##view_all_groups url
    view_all_groups_url = profile_url_complete + '/groups'
    r1 = requests.get(view_all_groups_url, headers= headers)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    groups_text= soup1.find_all('h3',{'class':'title'})
        
        #extracting names of groups to a list
    groups_text_list=[]
    for item_len in range(len(groups_text)):
      try:
        groups_text_profile= soup1.find_all('h3',{'class':'title'})[item_len].text#parameterize on i
        groups_text_list.append(groups_text_profile)
      except Exception as e:
        groups_text_profile = 'Content Not Available'
        groups_text_list.append(groups_text_profile)
            

    #extracting all_replies_url page  
    view_all_replies_url = profile_url_complete+ '/replies'
    r2 = requests.get(view_all_replies_url, headers= headers)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    all_replies= soup2.find_all('div',{'class':'col col--small-12 recent-list'})#[i].text#parameterize on i
        
    #total_count_of_available_replies
    total_count_of_available_replies= len(all_replies)
    print('total_count_of_available_replies', total_count_of_available_replies)
    
    
    
        
    temp_list=[profile_name_cleaned, profile_url_complete, view_all_groups_url, view_all_replies_url, groups_text_list, cleaned_join_date, total_posts, total_count_of_available_replies]
    full_list.append(temp_list)
        
df = pd.DataFrame(full_list,columns=("Profile Name","Profile URL", "Part of Groups URL","View All Replies URL","Is part of groups", "Joined Date", "Total_Posts_till_date", "available_replies"))
    

    

pd.set_option("max_colwidth",None)
df
#df_final = df[df['profile Name'] == df['author_info']]

In [ ]:
df.to_csv('profile_extract_part1.csv', index=False)

In [ ]:

full_final_list=[]
for index, row in df.iterrows():
    temp_list1 =[]
    

    if int(row['available_replies'])==0:
      
      reply_url= 'NA'
      response_text= 'NA'
      response_time='NA'
      temp_list1=[row["Profile Name"], row["Profile URL"], row["Part of Groups URL"], row["View All Replies URL"], row["Is part of groups"], row["Joined Date"], row["Total_Posts_till_date"], row["available_replies"], reply_url, response_text,response_time]
      full_final_list.append(temp_list1)

    else:
      
      headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
      Replies_URL= row['View All Replies URL']
      print('Replies_URL: ', Replies_URL)
      r = requests.get(Replies_URL, headers= headers)
      soup = BeautifulSoup(r.text, 'html.parser')
      
      response_count= soup.find_all('time',{'class': 'fuzzy'})
      print("response_count: ", response_count)
      for i in range(len(response_count)):
        reply_link= soup.find_all('div',{'class':'col col--small-12 recent-list'})[i].find_all('a')[1]['href']
        reply_url= 'https://patient.info'+reply_link
        print(reply_url)

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
        r = requests.get(reply_url, headers= headers)
        soup5 = BeautifulSoup(r.text, 'html.parser')

        print("i: ", i)
        lower_case_url= str.lower(reply_url)
        comment_id= lower_case_url.split('commentid=')[1]
        print('comment_id: ', comment_id)
        try:
          reply_text_1= soup5.find('article',{'id':comment_id}).find('input',{'class':'moderation-conent'})['value']
          reply_time_1= soup5.find('article',{'id':comment_id}).find('time',{'class':'fuzzy'})['datetime']
        except Exception as e:
          reply_text_1 = "NA"
          reply_time_1 = 'NA'
        # reply_text_1= soup5.find('article',{'id':comment_id}).find('input',{'class':'moderation-conent'})['value']
        response_text= reply_text_1
        response_time= reply_time_1

        temp_list1=[row["Profile Name"], row["Profile URL"], row["Part of Groups URL"], row["View All Replies URL"], row["Is part of groups"], row["Joined Date"], row["Total_Posts_till_date"], row["available_replies"], reply_url, response_time, response_text]
        full_final_list.append(temp_list1)


df_final = pd.DataFrame(full_final_list,columns=("Member_Username", "Profile URL", "Part of Groups URL", "View All Replies URL", "Is part of groups", "Joined Date", "Total_Posts_till_date", "available_replies_to_extract", "specific_reply_url", "response_time","response_text"))

In [ ]:
df_final.to_csv('profile_extract_finalized.csv', index= False)